In [2]:
import pandas as pd

import os 
PATH = '/home/politlen/Downloads/Telegram Desktop'
history = 'history.csv'
states = 'states.csv'

hist_df = pd.read_csv(os.path.join(PATH, history), converters={'history': eval})

In [ ]:
len(hist_df)

In [ ]:
import json
import datetime

pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

test_cell = hist_df.history.loc[0]

def get_updates(json):
    return [x for x  in json if x['type']=='update']
    

def to_dt(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d_%H:%M:%S')

def get_time_diff(json):
    return to_dt(json[-1]['ts']) - to_dt(json[0]['ts'])

def get_data(json):
    length = len(json)
    
    return length

print(get_data(test_cell))
# print(json.dumps(get_updates(test_cell), sort_keys=True, indent=4, ensure_ascii=False))
print(get_time_diff(test_cell))
hist_df['len'] = hist_df.history.apply(lambda x:len(x))
hist_df['updates'] = hist_df.history.apply(lambda x:len(get_updates(x)))
hist_df['hello'] = hist_df.history.apply(lambda x: len([i for i in x if i['action'] == 'hello']))
hist_df['start'] = hist_df.history.apply(lambda x: len([i for i in x if i['action'] == '/start']))

hist_df['time_diff'] = hist_df.history.apply(get_time_diff)
hist_df.columns

In [ ]:
# hist_df['steps_cut'] = pd.qcut(hist_df['steps'], q=3)
hist_df[hist_df.start > 3]

In [ ]:
hist_df.history.loc[5]

In [ ]:
low = hist_df[hist_df.time_diff < datetime.timedelta(seconds=30)]
print('Количество запией: ', len(low))
low.head()

In [ ]:
for i in low.history:
    print(json.dumps(i, sort_keys=True, indent=4, ensure_ascii=False), end = '**'*16)

In [63]:
def get_action_list(df):
    action_list = []
    for row in df:
        action_list.append([x['action'] for x in row if x['type'] == 'update'])
    
    return action_list

In [ ]:
from collections import Counter
cnt = Counter()

for el in get_action_list(hist_df.history):
     el = str(el)
     cnt[el] += 1
        

cnt.most_common(20)

In [ ]:
get_action_list(hist_df.history)[:2]

In [45]:
import numpy as np

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
#     print(matrix)
    return (matrix[size_x - 1, size_y - 1])



In [ ]:
lev = []
for seq in get_action_list(hist_df.history):
    if len(seq) == 1:
        lev.append(None)
    else:
        lev.append(levenshtein(seq, ['hello', 'help_callback']))
    
print(lev)

In [51]:
hist_df['lev'] = lev

In [ ]:
for i in hist_df[hist_df.lev<3].history:
    print(json.dumps([x['action'] for x in i if x['type']=='update'], sort_keys=True, indent=4, ensure_ascii=False), end = '\n')


In [ ]:
def to_dt(string):
    if '"' in string:
        string = string.replace('"', '')
    return datetime.datetime.strptime(string, '%Y-%m-%d_%H:%M:%S')

def get_last_date(days=1):
    """days устанавливает значения в днях для получения нужной даты"""
    current_time = datetime.datetime.today() 
    return current_time - datetime.timedelta(days=days)


def get_new_users(array, flag=0):
    """set 0 to get new users and -1 to get unique"""
    counter = 0
    for elem in array:
        date = to_dt([x['ts'] for x in elem][flag])
        if date>get_last_date(10):
            counter +=1
    return counter


def get_new_users_indexes(array, flag=0):
    """set 0 to get new users and -1 to get unique"""
    index_list = []
    for index, elem in enumerate(array):
        date = to_dt([x['ts'] for x in elem][flag])
        if date>get_last_date(10):
            index_list.append(index)
            
    return index_list

unique = get_new_users_indexes(hist_df.history, -1)
new = get_new_users_indexes(hist_df.history, 0)
old = set(unique).difference(set(new))
hist_df[hist_df['Unnamed: 0'].isin(unique)]
hist_df[hist_df['Unnamed: 0'].isin(new)]
hist_df[hist_df['Unnamed: 0'].isin(old)]

